In [23]:
import pandas as pd
import numpy as np

In [24]:
dt = pd.read_csv("TrainSet _1st.csv")
dt.columns = ['F','C','Y']
dt = dt.drop_duplicates(subset = None, keep= 'first', inplace= False, ignore_index= True)
dt.fillna("", inplace=True)
dt['New'] = dt['F'] + "\n" + dt['C']
dt1 = dt[['New', 'Y']]

dt1

,New,Y
0,Clinical information : 두부외상 후 후유증 평가\r\n\r\nAx...,0
1,Clinical information : lung cancer\nAxial T1WI...,0
2,Clinical information : Multiple Sclerosis\r\n\...,0
3,Clinical information : patient with DLBCL.\r\n...,0
4,Clinical information : Transient cerebral isch...,1
...,...,...
5331,Clinical information : s/p Removal of vestibul...,0
5332,"CI, headache of sudden onset (known UIA).\r\nA...",0
5333,Clinical information : patient with DLBCL.\n\n...,0
5334,Clinical information : Lung cancer patient 임.\...,0


In [25]:
dt_ch_text=dt1[dt1['New'].str.contains('no specific interval change compare to the latest MR brain', na=False)]
dt_ch_text[dt_ch_text['Y'] != 0].index

Int64Index([4892], dtype='int64')

In [26]:
dt_ch1_text=dt1[dt1['New'].str.contains('No evidence of acute infarction', na=False)]
dt_ch1_text[dt_ch1_text['Y'] != 0].index

Int64Index([217, 2720], dtype='int64')

In [27]:
dt1.drop([4892,217,2720], axis=0, inplace=True)

C:\Users\Byeonghoon\AppData\Local\Temp\ipykernel_6132\116378906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.drop([4892,217,2720], axis=0, inplace=True)


In [28]:
dt1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5333 entries, 0 to 5335
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   New     5333 non-null   object
 1   Y       5333 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 125.0+ KB


In [29]:
import string
import re
import nltk
from nltk.corpus import stopwords

In [30]:
def howmax(text):
  li = re.findall("\n[0-9]+[.] ", re.sub(" *\n *", "\n", text))
  li = [re.sub("[^\d+]", "", i) for i in li]

  if len(li) != 0:
    li = [int(i) for i in li]
    res = max(li)
    return res

num = max(dt1['New'].map(str).apply(lambda x:howmax(x)))

In [31]:
#\r, \n : 불필요한 문자
#Clinical information, CI이라는 단어가 불필요하다고 생각함 -> 삭제
dt1 = dt1.replace('\r', ' ', regex=True)
dt1 = dt1.replace('Clinical information','', regex=True)
dt1 = dt1.replace('CI','', regex=True)
dt1 = dt1.replace('조영증강은 시행하지 않았음', '', regex=True)

# 기호 제거
def clean_text(text):

  text_rmv = re.sub("[^a-zA-Z가-힣0-9\n +/.]", " ", text)
  text_rmv = re.sub("\n+", "\n", text_rmv)
  text_rmv = re.sub(" +", " ", text_rmv)
  text_rmv = ' '.join(text_rmv.split())

  return text_rmv

dt1['New'] = dt1['New'].map(str).apply(lambda x: clean_text(x))

In [32]:
#1~howmax(numm)까지 뒤에 . 붙여서 1. 2. 3. ,,,이렇게 만드는 과정
emlist = []
def delnum(bb):
  for i in range(1,int(bb)+1):
    soot = str(i) + str('. ')
    emlist.append(soot)
  return emlist

final_num_list = delnum(num)
final_num_list

def seee(daataa, f_num_l):

  for i in daataa.index.tolist():

    for k in f_num_l:
      daataa[i] = daataa[i].replace(k, ' ')
    
  return daataa

dt1['New'] = seee(dt1['New'], final_num_list)

C:\Users\Byeonghoon\AppData\Local\Temp\ipykernel_6132\2928146050.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daataa[i] = daataa[i].replace(k, ' ')


In [33]:
dt1['New'][552]

'Lung cancer Axial T1WI sagittal T1WI axial T2WI axial FLAIR axial T2 GRE image 획득하였으며 조영증강을 시행함.  No evidence of metastasis.  A enhancing mass with dark SI on SWI at Lt temporal convexity Calcified meningioma 7.1 x 9.0mm in size no change.  Rt mastoid effusion. No evidence of metastasis.'

In [9]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.add_dictionary(['조영증강', 'T1WI', 'T2WI', 'T1', 'T2'], 'Noun')

def tw_tokenizer(text):
  tokens_ko = twitter.morphs(text)
  return tokens_ko 

C:\anaconda3\envs\bhoon\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
# test train split
from sklearn.model_selection import train_test_split

X = dt1['New']
y = dt1['Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline

In [61]:
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,4),min_df=3, max_df=0.9)
tfidf.fit(X)
X_train_tfidf = tfidf.transform(X)

def logistic_tuning(train_sprs,y,param):
    model = LogisticRegression()
    grid = GridSearchCV(model, param, scoring='roc_auc', cv=3)
    grid.fit(train_sprs, y)
    
    print(grid.best_params_)
    print(grid.best_score_)
    
    return grid.best_estimator_

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [62]:
%%time
param1= {'C':[15,20,25],
        'max_iter': [50, 60, 70, 80, 90]}

logistic_tuning(X_train_tfidf, y, param = param1)

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

{'C': 25, 'max_iter': 80}
0.9955026071723845
CPU times: total: 1min 29s
Wall time: 15 s


LogisticRegression(C=25, max_iter=80)

In [89]:
tfidf = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,3),min_df=2, max_df=0.9)
tfidf.fit(X_train)

X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

lr = LogisticRegression(C=10, max_iter=80)
lr.fit(X_cnt, y_train)
pred = lr.predict(X_test_cnt)
pred_probs = lr.predict_proba(X_test_cnt)[:,1]

print('AUROC: {0: .6f}'.format(roc_auc_score(y_test, pred_probs)))

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


AUROC:  0.995863


In [68]:
#countervector
from sklearn.feature_extraction.text import CountVectorizer

cnt = CountVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,3),min_df=2, max_df=0.9)
cnt.fit(X)
X_cnt = tfidf.transform(X)

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [72]:
%%time
param2= {'C':[100,150,200,500],
        'max_iter': [50, 70,71,72,73]}

logistic_tuning(X_cnt, y, param = param2)

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

{'C': 200, 'max_iter': 70}
0.9956963349265848
CPU times: total: 2min 3s
Wall time: 21 s


C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=200, max_iter=70)

In [87]:
# countervector
cnt = CountVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,3),min_df=2, max_df=0.9)
cnt.fit(X_train)
X_cnt = tfidf.transform(X_train)
X_test_cnt = tfidf.transform(X_test)

lr = LogisticRegression(C=5, max_iter=70)
lr.fit(X_cnt, y_train)
pred = lr.predict(X_test_cnt)
pred_probs = lr.predict_proba(X_test_cnt)[:,1]

print('AUROC: {0: .6f}'.format(roc_auc_score(y_test, pred_probs)))

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


AUROC:  0.995427


In [14]:
# DL
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

print('최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))

최대 길이 : 383
평균 길이 : 44.749648


In [15]:
max_len = 383
X_train_padded = pad_sequences(X_train_encoded, maxlen= max_len)

In [16]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

In [25]:
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_padded, y_train, epochs=20, batch_size=64, validation_split=0.1)

Epoch 1/20
60/60 [==============================] - 3s 40ms/step - loss: 0.3849 - acc: 0.8677 - val_loss: 0.3467 - val_acc: 0.8759
Epoch 2/20
60/60 [==============================] - 2s 34ms/step - loss: 0.2608 - acc: 0.8924 - val_loss: 0.2310 - val_acc: 0.9110
Epoch 3/20
60/60 [==============================] - 2s 35ms/step - loss: 0.1820 - acc: 0.9252 - val_loss: 0.1791 - val_acc: 0.9180
Epoch 4/20
60/60 [==============================] - 2s 34ms/step - loss: 0.1317 - acc: 0.9497 - val_loss: 0.2223 - val_acc: 0.8923
Epoch 5/20
60/60 [==============================] - 2s 34ms/step - loss: 0.1065 - acc: 0.9659 - val_loss: 0.1122 - val_acc: 0.9555
Epoch 6/20
60/60 [==============================] - 2s 35ms/step - loss: 0.0754 - acc: 0.9789 - val_loss: 0.1299 - val_acc: 0.9602
Epoch 7/20
60/60 [==============================] - 2s 35ms/step - loss: 0.0873 - acc: 0.9737 - val_loss: 0.0940 - val_acc: 0.9719
Epoch 8/20
60/60 [==============================] - 2s 35ms/step - loss: 0.0517 - a

In [27]:
x_test2_encoded = tokenizer.texts_to_sequences(X_test)
x_test2_padded = pad_sequences(x_test2_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test2_padded, y_test)[1]))

34/34 [==============================] - 0s 9ms/step - loss: 0.1109 - acc: 0.9709

 테스트 정확도: 0.9709
